In [6]:
import Parity_Data_Generator
import numpy as np
from pyESN import ESN
import time

N = 7000   # number of datapoints
n = 3       # n-parity
rng = np.random.RandomState(42)



start_time = time.time()
# produce Data
bits, parity, target = Parity_Data_Generator.generateParityData(N, n, randomstate=rng)

# Divide in training and test data
traintest_cutoff = int(np.ceil(0.7 * len(bits)))
train_bits, test_bits = bits[:traintest_cutoff], bits[traintest_cutoff:]
train_parity, test_parity = parity[:traintest_cutoff], parity[traintest_cutoff:]
train_targets, test_targets = target[:traintest_cutoff], target[traintest_cutoff:]

print("--- %s seconds ---" % (time.time() - start_time))

In [7]:
def runSomeConfigs(spectral_rad, leak_r, train_bits, test_bits, train_parity, test_parity):
#     print("Spectral-radius:",spectral_rad, "leak-rate:", leak_r)
    nParityESN = ESN(n_inputs=1,
                          n_outputs=1,
                          n_reservoir=500,  # from 200 onwards test error decreases significantly
                          spectral_radius=spectral_rad,  # high spectral rad is better (high fluctuations 0->1->0)
                          leak_rate= leak_r,   # adjust leak_rate
                          sparsity=0.95,
                          noise=0.001,
                          input_shift=0,
                          input_scaling=3,  # 3 seems to be fine. Maybe use lower
                          teacher_scaling=1.12,
                          teacher_shift=-0.7,
                          out_activation=np.tanh,
                          inverse_out_activation=np.arctanh,
                          random_state=rng,
                          silent=False)
    # print(np.shape(bits),np.shape(parity),np.shape(target))
    nParityESN.fit(train_bits, train_parity)
    pred_test = nParityESN.predict(test_bits)

    #### not rounding
    predictedParity = pred_test # not rounding
    #### Rounding
#     better_pred_test = []           # Maybe don't round and return "true" results from pred_test
#     for x in pred_test:
#         better_pred_test.append(0 if x < 0.5 else 1)
#     predictedParity = np.array(better_pred_test).reshape(-1,1)
#     print(test_parity[40030:40100].reshape(-1))
#     print(predictedParity[40030:40100].reshape(-1))
    
    print("Testing error")
    test_error = np.sqrt(np.mean((predictedParity - test_parity) ** 2))
    print(test_error)
    return test_error

def produceTrainingData(randomState):
    bits, parity, target = Parity_Data_Generator.generateParityData(N, n, randomstate=np.random.RandomState(randomState))

    # Divide in training and test data
    traintest_cutoff = int(np.ceil(0.7 * len(bits)))
    train_bits, test_bits = bits[:traintest_cutoff], bits[traintest_cutoff:]
    train_parity, test_parity = parity[:traintest_cutoff], parity[traintest_cutoff:]
#     train_targets, test_targets = target[:traintest_cutoff], target[traintest_cutoff:]
    return train_bits, test_bits, train_parity, test_parity

In [8]:
trials = 10
# num_neurons = [500, 600, 700, 800, 900, 1000]
# num_neurons = [500]
n = 500
# num_neurons = [100, 200, 300, 400, 500, 600, 700, 800]
l_rate = np.linspace(0.1, 1, num=10)
# l_rate =[0.1]
# l_rate = [0.1,0.2,0.3]
# sp_rad = np.linspace(0.9, 1.0, num=5)
sp_rad = np.linspace(0.1, 1, num=10)
# sp_rad = [0.9,0.92,0.94,0.96,0.98, 1]

run_time = time.time()
iterations, iterationcount = trials*len(sp_rad)*len(l_rate), 0
errors = []
all_errors = [{} for _ in range(trials)] # empty dict
for i in range(trials):
    errors = []
    # produce training and testing data
    train_bits, test_bits, train_parity, test_parity = produceTrainingData(i)
    
    for l in l_rate:
        for s in sp_rad:
            print("Neurons:", n, "Spectral-radius:", s, "Leak-rate:", l)
            errors.append(runSomeConfigs(s,l, train_bits, test_bits, train_parity, test_parity))
            iterationcount = iterationcount + 1
            print("--- Progress: {:.2f}% --- {:.2f} minutes ---".format(iterationcount/iterations*100, ((time.time() - run_time)/60) ))
    ## Save leak-rate,spectral-radius and error in dictionary
    all_errors[i] = errors
            
print("Total time for Testbench:")
print("--- {:.2f} seconds ---".format(time.time() - run_time))
print("--- {:.2f} minutes ---".format(((time.time() - run_time)/60)))

Neurons: 500 Spectral-radius: 0.1 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.16795847495
Testing error
0.737055221884
--- Progress: 0.10% --- 0.28 minutes ---
Neurons: 500 Spectral-radius: 0.2 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.179196707992
Testing error
0.724590111676
--- Progress: 0.20% --- 0.52 minutes ---
Neurons: 500 Spectral-radius: 0.3 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.188918441297
Testing error
0.689753618398
--- Progress: 0.30% --- 0.76 minutes ---
Neurons: 500 Spectral-radius: 0.4 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.19511161275
Testing error
0.799662669771
--- Progress: 0.40% --- 1.01 minutes ---
Neurons: 500 Spectral-radius: 0.5 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.210272409207
Testing error
0.682305311981
--- Progress: 0.50% --- 1.25 minutes ---
Neurons: 500 Spectral-radius: 0.6 Leak-rate: 0.1
harvesting states...
fitting...
training er

In [13]:
import pickle
#### Save Data
# with open('datasaveAverageSlow.p', 'wb') as outputFile:
#     pickle.dump(all_errors, outputFile)
# #     pickle.dump(num_neurons, outputFile)
#     pickle.dump(trials, outputFile)
#     pickle.dump(l_rate, outputFile)
#     pickle.dump(sp_rad, outputFile)
#### Load Data
# with open('datasaveAverageSlow.p', 'rb') as inputFile:
#     all_errors = pickle.load(inputFile)
# #     num_neurons = pickle.load(inputFile)
#     trials = pickle.load(inputfile)
#     l_rate = pickle.load(inputFile)
#     sp_rad = pickle.load(inputFile)

In [46]:
### Average errors
errorLists = np.zeros(100)
for i in range(len(all_errors)):
    e = all_errors[i]
    errorLists = np.add(errorLists, np.array(e))

errorLists = np.divide(errorLists, trials)

<class 'numpy.ndarray'>
[ 0.71054866  0.75445873  0.77638588  0.77169606  0.74329063  0.6451233
  0.6148545   0.60735191  0.59799724  0.59724528  0.76716284  0.76257741
  0.73333803  0.7184355   0.76732732  0.70620888  0.64675575  0.63293593
  0.61942795  0.60861806  0.72013618  0.73551312  0.75757475  0.7927151
  0.74394836  0.71981378  0.68513855  0.65741974  0.62319707  0.61920016
  0.67844134  0.82538287  0.8249066   0.80035524  0.76823529  0.74350792
  0.69194268  0.65481667  0.63558957  0.63082156  0.64916848  0.69363213
  0.75005076  0.75129194  0.80522983  0.77942714  0.73492391  0.7370404
  0.67258286  0.63282603  0.59159617  0.65406085  0.67403915  0.75081341
  0.78238317  0.79162624  0.75806848  0.79439447  0.71263107  0.65390656
  0.58365066  0.60866335  0.67210219  0.69190112  0.73892493  0.75719139
  0.79784216  0.76814262  0.81568367  0.72726477  0.57378332  0.6142376
  0.63597697  0.68672908  0.72857736  0.75873651  0.73118368  0.80708434
  0.80577061  0.78564418  0.592

In [48]:
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('HansBambel', 'T0F76G8esWCDFNmIVEUv')

# c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(all_errors))]
# c= ["blue", "orange", "green", "cyan", "yellow", "magenta", "red", "black"]

# x, y, z = all_errors[0]
x = l_rate
y = sp_rad
z = errorLists
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    name=("Averaged over", len(trials) ,"Trials"),
    mode='markers',
    marker=dict(
        size=7,
#             color=c[i],                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
#     margin=dict(
#         l=0,
#         r=0,
#         b=0,
#         t=0
#     ),
#     go.zaxis=dict(
#         range=[0, 0.8]
#     ),
    scene=go.Scene(
        xaxis=go.XAxis(title='x = Leak-Rate'),
        yaxis=go.YAxis(title='y = Spectral Radius'),
        zaxis=go.ZAxis(title='z = Error')
        )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='3d-scatter-errors for different Reservoirsizes')

In [52]:
data = [
     go.Contour(
        z=errorLists.reshape(10,10),
        x=l_rate,
        y=sp_rad,
        colorscale='Jet',
    )]
py.iplot(data)

In [40]:
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('HansBambel', 'T0F76G8esWCDFNmIVEUv')

# c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(all_errors))]
# c= ["blue", "orange", "green", "cyan", "yellow", "magenta", "red", "black"]
data = []
for i, n in enumerate(all_errors):

    x, y, z = all_errors[i]

    trace = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        name=(str(num_neurons[i])+" Neurons"),
        mode='markers',
        marker=dict(
            size=7,
#             color=c[i],                # set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=0.8
        )
    )
    data.append(trace)

# data = [trace1]
layout = go.Layout(
#     margin=dict(
#         l=0,
#         r=0,
#         b=0,
#         t=0
#     ),
#     go.zaxis=dict(
#         range=[0, 0.8]
#     ),
    scene=go.Scene(
        xaxis=go.XAxis(title='x = Leak-Rate'),
        yaxis=go.YAxis(title='y = Spectral Radius'),
        zaxis=go.ZAxis(title='z = Error')
        )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='3d-scatter-errors for different Reservoirsizes')

KeyboardInterrupt: 

In [7]:
nParityESN = ESN(n_inputs=1,
                          n_outputs=1,
                          n_reservoir=500,  # from 200 onwards test error decreases significantly
                          spectral_radius=0.95,  # high spectral rad is better (high fluctuations 0->1->0)
                          leak_rate= 0.1,   # adjust leak_rate
                          sparsity=0.95,     # 0.95
                          noise=0.01,      # 0.001
                          input_shift=-0.5,    # 0
                          input_scaling=3,  # 3 seems to be fine. Maybe use lower
                          teacher_scaling=1.12,
                          teacher_shift=-0.7,
                          out_activation=np.tanh,
                          inverse_out_activation=np.arctanh,
                          random_state=rng,
                          silent=False)
nParityESN.fit(train_bits, train_parity)

harvesting states...
fitting...
training error:
0.22566015585


array([[ 0.63391576],
       [-0.14456233],
       [-0.15365048],
       ..., 
       [ 0.88192822],
       [ 0.88365077],
       [ 0.86273787]])

In [11]:
##### Create Training Data for fastESN with output from slowESN
start_time = time.time()

new_rng = np.random.RandomState(1337)
new_N, new_n = 5000, 3
new_bits, new_parity, new_target = Parity_Data_Generator.generateParityData(new_N, new_n, randomstate=new_rng)

# Divide in training and test data
traintest_cutoff = int(np.ceil(0.7 * len(bits)))
new_train_bits, new_test_bits = new_bits[:traintest_cutoff], new_bits[traintest_cutoff:]
# new_train_parity, new_test_parity = new_parity[:traintest_cutoff], new_parity[traintest_cutoff:]
new_train_targets, new_test_targets = new_target[:traintest_cutoff], new_target[traintest_cutoff:]

train_predicted_parity = nParityESN.predict(new_train_bits)
test_predicted_parity = nParityESN.predict(new_test_bits)

print("--- %s seconds ---" % (time.time() - start_time))

--- 9.439780473709106 seconds ---


In [14]:
#### FastESN ####
def runSomeFastConfigs(spectral_rad, leak_r,num_neurons=100):
    outputESN = ESN(n_inputs=1,
                              n_outputs=1,
                              n_reservoir=num_neurons,  
                              spectral_radius=spectral_rad,  # high spectral rad is better (high fluctuations 0->1->0)
                              leak_rate= leak_r,   # adjust leak_rate
                              sparsity=0.95,
                              noise=0.001,
                              input_shift=0,
                              input_scaling=3,  # 3 seems to be fine. Maybe use lower
                              teacher_scaling=1.12,
                              teacher_shift=-0.4,
                              out_activation=np.tanh,
                              inverse_out_activation=np.arctanh,
                              random_state=new_rng,
                              silent=False)
        # print(np.shape(bits),np.shape(parity),np.shape(target))
    outputESN.fit(train_predicted_parity, new_train_targets)
    pred_test = outputESN.predict(test_predicted_parity)
        
    print("Testing error")
    test_error = np.sqrt(np.mean((pred_test - new_test_targets) ** 2))
    print(test_error)
    return test_error

In [15]:
num_neurons_fast = [100]
# l_rate_fast = np.linspace(0.4, 1, num=7)
l_rate_fast = np.linspace(0.1, 1, num=10)
# l_rate_fast = [0.6]
# sp_rad_fast = np.linspace(0.8, 1, num=3)
sp_rad_fast = np.linspace(0.1, 1, num=10)
# sp_rad_fast = [0.95]

run_time = time.time()
iterations, iterationcount = len(num_neurons_fast)*len(sp_rad_fast)*len(l_rate_fast), 0
leak_rate_list = []
spectral_radius_list = []
errors = []
all_errors_fast = [{} for _ in range(len(num_neurons_fast))]
for i,n in enumerate(num_neurons_fast):
    leak_rate_list = []
    spectral_radius_list = []
    errors = []
    for l in l_rate_fast:
        for s in sp_rad_fast:
            print("Neurons:", n, "Spectral-radius:", s, "Leak-rate:", l)
            leak_rate_list.append(l)
            spectral_radius_list.append(s)
            errors.append(runSomeFastConfigs(s,l,n))
            iterationcount = iterationcount + 1
            print("--- Progress: {:.2f}% --- {:.2f} minutes ---".format(iterationcount/iterations*100, ((time.time() - run_time)/60) ))
    ## Save leak-rate,spectral-radius and error in dictionary
    all_errors_fast[i] = leak_rate_list, spectral_radius_list, errors
            
print("Total time for Testbench:")
print("--- {:.2f} seconds ---".format(time.time() - run_time))
print("--- {:.2f} minutes ---".format(((time.time() - run_time)/60)))

Neurons: 100 Spectral-radius: 0.1 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.101186828
Testing error
0.12524643894
--- Progress: 1.00% --- 0.06 minutes ---
Neurons: 100 Spectral-radius: 0.2 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.103869287379
Testing error
0.125047991269
--- Progress: 2.00% --- 0.12 minutes ---
Neurons: 100 Spectral-radius: 0.3 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.107079482285
Testing error
0.125719654047
--- Progress: 3.00% --- 0.18 minutes ---
Neurons: 100 Spectral-radius: 0.4 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.109504150843
Testing error
0.125833776198
--- Progress: 4.00% --- 0.24 minutes ---
Neurons: 100 Spectral-radius: 0.5 Leak-rate: 0.1
harvesting states...
fitting...
training error:
0.10798566413
Testing error
0.125259035503
--- Progress: 5.00% --- 0.30 minutes ---
Neurons: 100 Spectral-radius: 0.6 Leak-rate: 0.1
harvesting states...
fitting...
training error

In [16]:
import pickle
#### Save Data
with open('datasaveFast.p', 'wb') as outputFile:
    pickle.dump(all_errors_fast, outputFile)
    pickle.dump(num_neurons_fast, outputFile)
    pickle.dump(l_rate_fast, outputFile)
    pickle.dump(sp_rad_fast, outputFile)
#### Load Data
# with open('datasaveFast.p', 'rb') as inputFile:
#     all_errors_fast = pickle.load(inputFile)
#     num_neurons_fast = pickle.load(inputFile)
#     l_rate_fast = pickle.load(inputFile)
#     sp_rad_fast = pickle.load(inputFile)

In [18]:
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('HansBambel', 'T0F76G8esWCDFNmIVEUv')

# c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(all_errors_fast))]
c= ["blue", "orange", "green", "cyan", "yellow", "magenta", "red", "black"]
data = []
for i, n in enumerate(all_errors_fast):

    x, y, z = all_errors_fast[i]

    trace = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        name=(str(num_neurons_fast[i])+" Neurons"),
        mode='markers',
        marker=dict(
            size=7,
#             color=c[i],                # set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=0.8
        )
    )
    data.append(trace)

# data = [trace1]
layout = go.Layout(
#     margin=dict(
#         l=0,
#         r=0,
#         b=0,
#         t=0
#     ),
#     go.zaxis=dict(
#         range=[0, 0.8]
#     ),
    scene=go.Scene(
        xaxis=go.XAxis(title='x = Leak-Rate'),
        yaxis=go.YAxis(title='y = Spectral Radius'),
        zaxis=go.ZAxis(title='z = Error')
        )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='3d-scatter-errors for different Reservoirsizes')